In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
import ipywidgets as widgets
from IPython.display import display, clear_output

# Generating synthetic data
np.random.seed(0)

# Create a DataFrame with expenses and incomes
descriptions = ['Groceries', 'Rent', 'Utilities', 'Entertainment', 'Salary', 'Transportation']
descriptions *= 17  # This will create 102 items

# Trim the list to match the number of periods
descriptions = descriptions[:100]

data = pd.DataFrame({
    'Date': pd.date_range(start='7/17/2024', periods=100),
    'Description': descriptions,
    'Amount': np.random.uniform(500, 6000, 100),  # Ensure this has 100 elements
    'Type': ['Expense', 'Income'] * 50
})

# Adding categories
categories = {
    'Groceries': 'Food',
    'Rent': 'Housing',
    'Utilities': 'Bills',
    'Entertainment': 'Leisure',
    'Salary': 'Work',
    'Transportation': 'Work'
}
data['Category'] = data['Description'].map(categories)
data['Day'] = data['Date'].dt.dayofweek
data['Month'] = data['Date'].dt.month
data['Year'] = data['Date'].dt.year

# Save the synthetic dataset to a CSV file
data.to_csv('synthetic_data.csv', index=False)

# Display the first few rows of the data
display(data.head())

# Data Preprocessing
features = data[['Amount', 'Day', 'Month', 'Year']]
target = data['Type'].apply(lambda x: 1 if x == 'Income' else 0)  # Binary encoding for income and expense

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

# Standardizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Training
model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_train_scaled, y_train)

# Making predictions
y_pred = model.predict(X_test_scaled)

# Evaluating the model
mse = mean_squared_error(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Classification Report: \n{report}')

# Interactive widgets for data input and visualization
def add_data(description, amount, date, entry_type):
    new_data = pd.DataFrame({
        'Date': [pd.to_datetime(date)],
        'Description': [description],
        'Amount': [amount],
        'Type': [entry_type]
    })
    new_data['Category'] = new_data['Description'].map(categories)
    new_data['Day'] = new_data['Date'].dt.dayofweek
    new_data['Month'] = new_data['Date'].dt.month
    new_data['Year'] = new_data['Date'].dt.year
    global data
    data = pd.concat([data, new_data], ignore_index=True)
    display_data()

def display_data():
    clear_output()
    display(description_widget, amount_widget, date_widget, type_widget, button, display_button, section_dropdown)
    display(data.tail())

def visualize_data(section):
    clear_output()
    display(description_widget, amount_widget, date_widget, type_widget, button, display_button, section_dropdown)
    if section == 'Weekly':
        data['Week'] = data['Date'].dt.isocalendar().week
        weekly_totals = data.groupby('Week')['Amount'].sum().reset_index()
        sns.lineplot(x='Week', y='Amount', data=weekly_totals)
        plt.xlabel('Week')
        plt.ylabel('Total Amount')
        plt.title('Weekly Total Amount')
    elif section == 'Monthly':
        monthly_totals = data.groupby('Month')['Amount'].sum().reset_index()
        sns.barplot(x='Month', y='Amount', data=monthly_totals)
        plt.xlabel('Month')
        plt.ylabel('Total Amount')
        plt.title('Monthly Total Amount')
    elif section == 'Quarterly':
        data['Quarter'] = data['Date'].dt.quarter
        quarterly_totals = data.groupby('Quarter')['Amount'].sum().reset_index()
        sns.barplot(x='Quarter', y='Amount', data=quarterly_totals)
        plt.xlabel('Quarter')
        plt.ylabel('Total Amount')
        plt.title('Quarterly Total Amount')
    elif section == 'Yearly':
        yearly_totals = data.groupby('Year')['Amount'].sum().reset_index()
        sns.barplot(x='Year', y='Amount', data=yearly_totals)
        plt.xlabel('Year')
        plt.ylabel('Total Amount')
        plt.title('Yearly Total Amount')
    plt.show()

description_widget = widgets.Text(value='', placeholder='Description', description='Description:', disabled=False)
amount_widget = widgets.FloatText(value=0, description='Amount:', disabled=False)
date_widget = widgets.DatePicker(description='Date', disabled=False)
type_widget = widgets.Dropdown(options=['Expense', 'Income'], value='Expense', description='Type:', disabled=False)
button = widgets.Button(description='Add Data')
display_button = widgets.Button(description='Display Data')
section_dropdown = widgets.Dropdown(options=['Weekly', 'Monthly', 'Quarterly', 'Yearly'], value='Monthly', description='Section:', disabled=False)

def on_button_clicked(b):
    add_data(description_widget.value, amount_widget.value, date_widget.value, type_widget.value)

def on_display_button_clicked(b):
    visualize_data(section_dropdown.value)

button.on_click(on_button_clicked)
display_button.on_click(on_display_button_clicked)

display(description_widget, amount_widget, date_widget, type_widget, button, display_button, section_dropdown)

# Initial display of data
display_data()


Text(value='', description='Description:', placeholder='Description')

FloatText(value=0.0, description='Amount:')

DatePicker(value=None, description='Date')

Dropdown(description='Type:', options=('Expense', 'Income'), value='Expense')

Button(description='Add Data', style=ButtonStyle())

Button(description='Display Data', style=ButtonStyle())

Dropdown(description='Section:', index=1, options=('Weekly', 'Monthly', 'Quarterly', 'Yearly'), value='Monthly…

,Date,Description,Amount,Type,Category,Day,Month,Year
95,2024-10-20,Transportation,1507.552491,Income,Work,6,10,2024
96,2024-10-21,Groceries,3725.821141,Expense,Food,0,10,2024
97,2024-10-22,Rent,610.591504,Income,Housing,1,10,2024
98,2024-10-23,Utilities,5059.170161,Expense,Bills,2,10,2024
99,2024-10-24,Entertainment,525.825119,Income,Leisure,3,10,2024
